In [136]:
#!/usr/bin/env python
import numpy as np
from math import ceil, floor
from numpy.linalg import det, inv, matrix_rank
# from sympy import Matrix, symbols, EmptySet
# from sympy.solvers.solveset import linsolve
from copy import copy
from cvxopt.glpk import ilp
from cvxopt import matrix
from itertools import combinations

primal = [2, 3, 5, 7, 11, 13, 17]

def get_hermit(dimensions=6, max_minor = 5):
    diag = np.array([1]*(dimensions)).astype('i')
    diag[-1] = max_minor
    pre_last = np.random.randint(max_minor, size=dimensions - 1)
    hermit = np.diag(diag)
    hermit[dimensions-1][:-1] = pre_last
    return hermit


def apply_restrictor(hermit, dimensions=6, max_minor = 5):
    continue_flag = True
    restricted = 0
    while continue_flag:
        restricted = np.append(hermit, np.random.randint(-max_minor, max_minor + 1, size=(1, dimensions)), 0)
        minors = get_minors(restricted)
        continue_flag = not all(minors)
    abs_minors = [abs(m) for m in minors]
    return restricted, min(abs_minors), max(abs_minors)


def minor(arr, i=0):  #  ith row removed
    return arr[np.array(range(i)+range(i+1, arr.shape[0]))[:, np.newaxis],
               np.array(range(arr.shape[1]))]


def get_minors(A):
    return [det(minor(A,i)) for i in xrange(A.shape[0])]
    
    
def is_valid_solution(solution):
    return  any(x for x in solution)


def generate_vertices(cone, b):
    vertices = []
    for i in xrange(cone.shape[0]):
        vertices.append(inv(minor(cone,i)).dot(b[range(i)+range(i+1, b.shape[0])]))
    return vertices 


def full_cycle(result_set, dimensions=6, p = 5):
    np.random.seed()
    h = get_hermit(dimensions, p)
    cone, min_m, max_m = apply_restrictor(h, dimensions, p)
#     print cone

    c = cone[-1]
    x = np.zeros(shape=(dimensions,1), dtype=float)
    x[-1] = float(dimensions)/float(p)

    sigma = c.dot(x)  # ( b[n+1])

    f = copy(x)#np.reshape(np.append(x, sigma), newshape=(dimensions + 1, 1))
    f[-1] = float(dimensions + 1)/float(p)

    # print 'f (vector-indicator`s last value): %s' % f[-1]  #  == sol
    
    is_in_M = c.dot(f) > sigma
#     if c.dot(f) > sigma:
#         print 'f belongs to M={x: cx >= %s}' % sigma
#     else:
#         print 'f doesn`t belong to M={x: cx >= %s}'% sigma

    target_f = matrix(-np.ones((dimensions + 1,1),dtype=float), tc='d')
    
    # cycle for changing sigma
#     s = None
#     if is_in_M:
#         sigma = floor(sigma)
#     else:
#         sigma = ceil(sigma)
#     while not s:        
    (status,s) = ilp(target_f, 
                 G=matrix(-cone, tc='d'),
                 h=matrix(-np.append(f, sigma), tc='d'),
                 I=set(range(dimensions)),
                )
#         if is_in_M:
#             sigma += 1
#         else:
#             sigma -= 1

    
    if status == 'LP relaxation is primal infeasible':
        print 'Primal infeasible'
        return
#     if is_valid_solution(s):
#         for s_s in s:
#             print float(s_s)
#     else:
#         print 'Trivial solution.'
#         for s_s in s:
#             print float(s_s)

    # print matrix_rank(cone)
    # print cone
    b = np.append(f, sigma)
    max_b = max(b)
    # print 'dots\n'

    # for i in xrange(cone.shape[0]):
    #     print inv(minor(cone,i)).dot(b[range(i)+range(i+1, b.shape[0])])
    vv = generate_vertices(cone, b)

    width_list = []
    for i,j in combinations(xrange(len(vv)), 2):
    #     print(pair)
        target_f2 = vv[i] - vv[j]
        restrictions = np.concatenate((inv(minor(cone, i).T), -inv(minor(cone, j).T), np.diag(np.ones(len(vv[0])).astype('d'))))
    #     print 'tar'
    #     print matrix(target_f2)
    # #     print restrictions
    #     print 'res'
    #     print matrix(-restrictions, tc='d')
    #     print 'ri'
    #     print matrix(np.zeros(shape=(restrictions.shape[0],1)), tc='d'),
        (status,s) = ilp(matrix(target_f2), 
             G=matrix(-restrictions, tc='d'),
             h=matrix(np.ones(shape=(restrictions.shape[0],1), dtype=np.double), tc='d'),
    # #                  A=matrix(1., (0,6)),b=matrix(1., (0,1)),
             I=set(range(len(vv)-1)),
    # #                  B=set()
            )
    #     print status, is_valid_solution(s)
        if status == 'optimal':
            width_c = abs(target_f2.dot(s)[0])
            if is_valid_solution(s) and width_c > float(0.0000):
                width_list.append(width_c)
            else:
                return
        else:
            return

#     print 'width: ', float(min(width_list))
#     print 'dimensions: ', dimensions
#     print 'mmin: ', min_m, ' mmax: ',max_m
    result_set.append( ( float(min(width_list)), dimensions, int(min_m), int(max_m), max_b ) )



from multiprocessing import Process, Value, Array, Manager
import time

# full_cycle(4,3)
results = None
manager = Manager()
results = manager.list()

while len(results) < 10:
    p = Process(target=full_cycle, args=(results, 10, 7))
    p.start()
    p.join(20)
    if p.is_alive():
        print "running... let's kill it..."
        # Terminate
        p.terminate()
        p.join()

for result in results:
    print(result)


(6.508914207449897e-17, 10, 6, 83, 1.5714285714285714)
(8.723180907769086e-17, 10, 4, 51, 7.1428571428571432)
(1.427429603089487e-16, 10, 1, 72, 7.1428571428571432)
(1.2040816326530608, 10, 4, 38, 1.5714285714285714)
(1.4538634846281803e-17, 10, 3, 59, 1.5714285714285714)
(1.093921887536462e-17, 10, 0, 51, 1.5714285714285714)
(1.0837891430864622e-16, 10, 6, 79, 1.5714285714285714)
(6.938893903907228e-18, 10, 0, 44, 1.5714285714285714)
(1.3869604561491144e-17, 10, 6, 56, 10.0)
(1.7840509870624062e-16, 10, 6, 65, 8.5714285714285712)


In [145]:
with open( 'res.csv', 'w' ) as f:
    for r in results:
        f.write(';'.join([str(l) for l in r]) + '\r\n')